<a href="https://colab.research.google.com/github/wanderson42/Portfolio-DS/blob/main/Deploy_Projeto_Dash_App_BigMartSales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# *Um "deploy" do projeto big mart sales prediction*

</div>
<center><img src="https://raw.githubusercontent.com/wanderson42/Portfolio-DS/main/datasets/Big_Mart_Sales/plotly_dash_logo.png" alt="Someone's feet on table facing a television"width="1800"/> <center> </div>
<p><center></center></p>


</div>
<center><img src="https://raw.githubusercontent.com/wanderson42/Portfolio-DS/main/datasets/Big_Mart_Sales/dataset-cover.png" alt="Someone's feet on table facing a television"width="1800"/> <center> </div>
<p><center></center></p>

Nesta aplicação dash plotly é possível efetuar previsões de vendas de um determinado produto das lojas da rede "Big Mart", levando em consideração uma série de atributos pré definidos. Além disso, considerando um conjunto de dados, é possível explorar esses dados de forma dinâmica, fazer multiplas previsões e compartilhar insights de forma fácil e interativa. Execute as células a seguir para preparar um ambiente jupyter notebook que torne possível executar a aplicação.   

In [8]:
import os
# Importando o modelo treinado via pkl
path = '/content/bigmart_dash_app/xgboost_model.joblib'
PathExist = os.path.exists(path)
if PathExist == True:
   %cd /content/bigmart_dash_app
else:
   %cd /content
   !git clone https://github.com/wanderson42/bigmart_dash_app.git
   %cd /content/bigmart_dash_app

/content/bigmart_dash_app


In [9]:
# Essas bibliotecas são essenciais para rodar um dash app no colab
%%capture
!pip install jupyter-dash
!pip install dash-bootstrap-components

In [27]:
import numpy as np
import re
import pandas as pd
import base64, io
import os
from urllib.parse import quote
from joblib import load
import dash
import dash_bootstrap_components as dbc
from dash import html, dcc, dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy.sparse import csr_matrix
import plotly.graph_objects as go
import plotly.express as px


A seguir estão as células que contem os arquivos da aplicação dash que originalmente foram escritas no vscode. A célula do arquivo helper.py contém funções auxiliares para o arquivo principal do app.py. Execute as células a seguir para acessar o dasboard através de uma url local!

**helper.py**

In [35]:
# @title
def text_intro():
    conteudo_inicio = """
    #### Sobre o Conjunto de Dados
    **O Big Mart Sales** é um conjunto de dados popular utilizado em problemas de previsão de vendas no varejo. Ele contém informações sobre vendas de produtos de várias lojas da rede "Big Mart", que estão localizadas em diferentes cidades e possuem diferentes tamanhos e tipos.

    O conjunto de dados é composto por duas partes: treinamento e teste. A parte de treinamento contém informações sobre 8523 produtos de 1559 lojas, enquanto a parte de teste contém informações sobre 5681 produtos de 1559 lojas. Cada entrada no conjunto de dados contém informações sobre o identificador do produto, o identificador da loja, a localização da loja, o tipo de produto, o peso e o preço do produto, entre outras informações.


    **Descrição dos atributos informativos:**

    - **Item_Weight:** Peso do produto;
    - **Item_Fat_Content:** Se o produto é com baixo teor de gordura ou não;
    - **Item_Visibility:** A porcentagem da área total de exposição de todos os produtos em uma loja alocada para um produto específico;
    - **Item_Type:** Denota o tipo de categoria que o produto pertence de forma mais ampla;
    - **Item_Reference:** Semelhante ao atributo Item_Type, contudo fornece uma categorização mais geral;
    - **Item_MRP:** Preço máximo de varejo (preço de lista) do produto;
    - **Outlet_Identifier:** ID único da loja;
    - **Outlet_Establishment_Year:** O ano em que a loja foi fundada;
    - **Outlet_Size:** O tamanho da loja em termos de área terrestre coberta;
    - **Outlet_Location_Type:** O tipo de cidade em que a loja está localizada;
    - **Outlet_Type:** Se a loja é apenas uma mercearia ou algum tipo de supermercado;


    - **Item_Outlet_Sales:** Representa as vendas totais de um produto específico em uma loja específica. Essa é a variável que estamos interessados em prever no conjunto de dados.


    Nesse contexto, este presente projeto tem como objetivo utilizar a biblioteca Dash para criar uma aplicação web interativa e personalizada para visualização e análise do Big Mart Sales Prediction Dataset. De forma a explorar os dados de forma dinâmica, fazer previsões e compartilhar insights de forma fácil e interativa.

    """

    return conteudo_inicio



# Função para criar menu suspenso de categorias da página "Previsões Individuais"
def create_dropdown(feature):
    # Dicionários com as categorias para as features discretas da página "Previsões"
    categorias = {
        "Outlet_Identifier": ['OUT010', 'OUT013', 'OUT017', 'OUT018', 'OUT019', 'OUT027', 'OUT035', 'OUT045', 'OUT046', 'OUT049'],
        "Item_Fat_Content": ['Low_Fat', 'Regular', 'Inedible'],  # Atualizado com as categorias corretas
        "Item_Reference": ['Drinks', 'Food', 'Non-Consumable'],
        "Item_Type": ['Dairy', 'Soft_Drinks', 'Meat', 'Fruits_and_Vegetables', 'Household', 'Baking_Goods', 'Snack_Foods', 'Frozen_Foods', 'Breakfast', 'Health_and_Hygiene', 'Hard_Drinks', 'Canned', 'Breads', 'Starchy_Foods', 'Others', 'Seafood'],  # Atualizado com as categorias corretas
        "Outlet_Size": ['Medium', 'Small', 'High'],  # Atualizado com as categorias corretas
        "Outlet_Location_Type": ['Tier_1', 'Tier_3', 'Tier_2'],  # Atualizado com as categorias corretas
        "Outlet_Type": ['Supermarket_Type1', 'Supermarket_Type2', 'Grocery_Store', 'Supermarket_Type3']  # Atualizado com as categorias corretas
    }
    return html.Div([
        html.H6(f"{feature}:"),
        dcc.Dropdown(
            id=f'dropdown-{feature}',
            options=[{'label': i, 'value': i} for i in categorias[feature]],
            placeholder=f"Selecione {feature}..."
        ),
    ])

# função para processar o arquivo .csv após o upload e exibi-lo em uma tabela de dados no callback:
def read_uploaded_data(contents):
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    loaded_data = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
    return loaded_data

def parse_contents(contents):
    loaded_data = read_uploaded_data(contents)

    return html.Div([
        html.H5('Dados do arquivo .csv:'),
        dash_table.DataTable(
            data=loaded_data.to_dict('records'),
            columns=[{'name': col, 'id': col} for col in loaded_data.columns],
            style_table={'height': '300px', 'overflowY': 'auto', 'position': 'relative'},
        ),
        html.Button('Fazer Previsões', id='submit-button', n_clicks=0, style={
            'background-color': '#f1863d',
            'border-radius': '5px',
            'padding': '5px',
            'color': 'black',
            'bottom': '5px',
        }),
    ], style={'width': '100%', 'position': 'relative'})



# Feature Enginering para novas previsões
def feature_engineering_preds(data, le, ohe):

    if isinstance(data, dict):
       # Cria um DataFrame com uma única linha de dados (uma previsão)
       data = pd.DataFrame([data])


    # Colunas categóricas para uma codificação via LabelEncoder
    le_categ_features = ['Outlet_Location_Type', 'Outlet_Type', 'Outlet_Size']

    # Ordenando as classes de Outlet_Size para o LabelEncoder
    data['Outlet_Size'].replace({'Small': 'A_Small', 'Medium': 'B_Medium', 'High': 'C_High'}, inplace=True)

    # Aplicar LabelEncoder às colunas especificadas
    for col in le_categ_features:
        data[col] = le.fit_transform(data[col])

    # Colunas categóricas para o OneHotEncoder
    ohe_categ_features = data.select_dtypes(include=['object']).columns.difference(le_categ_features)

    # Aplicando OneHotEncoder e obtendo uma matriz esparsa
    data_encoded_spmatrix = ohe.transform(data)

    return data_encoded_spmatrix


# Função para fazer previsões com base nas entradas do usuário.
def make_predictions(data):
    xgboost_model = load('/content/bigmart_dash_app/xgboost_model.joblib') # Carregar modelo treinado
    le = load('/content/bigmart_dash_app/label_encoder.pkl') # Carregar o objeto le
    ohe = load('/content/bigmart_dash_app/one_hot_encoder.pkl') # Carregar o objeto ohe
    # Aplicar a feature_engineering aos dados de entrada
    data = feature_engineering_preds(data, le, ohe)

    # Faça previsões usando o modelo treinado
    predictions = xgboost_model.predict(data)

    return predictions

def plotly_sales_by_category(data_before_encoded, target):
    # concatenar as features com a coluna das previsões "target"
    df = pd.concat([data_before_encoded, target], axis=1)
    # Calculando as porcentagens de vendas
    total_sales = df['Item_Outlet_Sales'].sum()
    df['Sales Percentage'] = df['Item_Outlet_Sales'] / total_sales * 100

    # Ordenando as categorias por porcentagem de vendas (ou outro critério desejado)
    df = df.sort_values('Sales Percentage', ascending=False)

    # Criando um gráfico de anel (donut chart) com setores de largura uniforme
    fig = go.Figure()

    fig.add_trace(go.Pie(
        labels=df['Item_Type'],
        values=df['Sales Percentage'],
        hole=0.7,  # Controla o tamanho do buraco no meio do anel
        textinfo='percent',
        insidetextfont=dict(color='black'),  # Define a cor do texto dentro das fatias
        marker=dict(colors=px.colors.qualitative.Set3, line=dict(color='white', width=2)),  # Estiliza as cores e adiciona bordas às fatias
    ))

    fig.update_traces(hole=0.6)  # Define o tamanho uniforme para o anel

    # Adicionando uma imagem dentro do buraco da pizza (cifrão de dinheiro)
    fig.add_layout_image(
        source="https://raw.githubusercontent.com/wanderson42/bigmart_dash_app/main/264-2640171_inventory-icon-white.png",  # URL da imagem do cifrão
        x=0.5,
        y=0.5,
        xanchor="center",
        yanchor="middle",
        sizex=0.4,  # Tamanho horizontal da imagem
        sizey=0.4,  # Tamanho vertical da imagem
    )

    fig.update_layout(
        title='Distribuição de Vendas por Categoria',
        title_x=0.15,  # Centraliza o título horizontalmente
        title_font=dict(color='black', size=20),  # Define a cor do texto do título como preto e o tamanho da fonte
        legend=dict(orientation="v", yanchor="middle", y=0.5, xanchor="left", x=1.2),  # Posiciona a legenda ao lado direito do anel
        margin=dict(t=50),  # Ajusta a margem superior para aproximar o título do gráfico
    )


    return fig

'''
def plotly_sales_over_time(data_before_encoded, target):
    # Agrupar os dados por categoria e ano/mês
    df = pd.concat([data_before_encoded, target], axis=1)
    sales_by_category = df.groupby(['Item_Type', 'Outlet_Type']).agg({'Item_Outlet_Sales': 'sum'}).reset_index()

    # Gráfico de linhas para cada categoria ao longo dos anos
    fig_line = px.line(sales_by_category, x='Outlet_Type', y='Item_Outlet_Sales', color='Item_Type',
                    title='Previsão de Vendas com base no tipo de loja')

    fig_line.update_layout(
        xaxis_title='Tipo de loja',
        yaxis_title='Venda Total',
        legend_title='Tipo de produto',
        #width=1200,
        height=600,
        title_x=0.5,  # Centraliza o título horizontalmente
        title_font=dict(color='black', size=20),
        plot_bgcolor='rgba(0,0,0,0)',  # Define o fundo do gráfico como transparente
        legend=dict(orientation='h', y=1.1),  # Posiciona a legenda acima do gráfico
        hovermode='x',  # Exibe informações no hover apenas na posição x
        hoverlabel=dict(bgcolor='white', font_size=12),  # Define o estilo do hover
        yaxis=dict(showgrid=True, gridcolor='lightgray'),  # Adiciona linhas de grade ao eixo y

    )

    fig_line.update_traces(
        mode='lines+markers',  # Adiciona marcadores aos pontos de dados
        line=dict(width=2),  # Ajusta a largura das linhas
        marker=dict(size=6),  # Define o tamanho dos marcadores
    )

    return fig_line
'''

import plotly.express as px

def plotly_sales_over_time(data_before_encoded, target, top_n=10):
    # Agrupar as features com a previsão
    df = pd.concat([data_before_encoded, target], axis=1)
    sales_by_category = df.groupby(['Item_Type', 'Outlet_Type']).agg({'Item_Outlet_Sales': 'sum'}).reset_index()

    # Calcular a soma total de vendas para cada tipo de produto
    total_sales_by_item = sales_by_category.groupby('Item_Type')['Item_Outlet_Sales'].sum().reset_index()
    total_sales_by_item = total_sales_by_item.sort_values(by='Item_Outlet_Sales', ascending=False)
    sorted_item_types = total_sales_by_item['Item_Type'].tolist()

    # Filtrar para incluir apenas os top 10 produtos mais vendidos em cada tipo de loja
    top_items = (
        sales_by_category
        .sort_values(by='Item_Outlet_Sales', ascending=False)
        .groupby('Outlet_Type')
        .head(top_n)
    )

    # Gráfico de barras agrupadas para cada produto em função das lojas
    fig_bar_grouped = px.bar(top_items, x='Outlet_Type', y='Item_Outlet_Sales', color='Item_Type',
                             title=f'Top {top_n} Produtos Mais Vendidos por Tipo de Loja',
                             labels={'Item_Outlet_Sales': 'Venda Total', 'Outlet_Type': 'Tipo de Loja'},
                             category_orders={'Item_Type': sorted_item_types},
                             barmode='group')  # Define o modo de barras agrupadas

    fig_bar_grouped.update_layout(
        xaxis_title='Tipo de loja',
        yaxis_title='Venda Total',
        legend_title='Tipo de produto',
        #width=1200,
        height=600,
        title_x=0.5,  # Centraliza o título horizontalmente
        title_font=dict(color='black', size=20),
        plot_bgcolor='rgba(255, 255, 255, 0.8)',  # Define o fundo do gráfico como branco semi-transparente
        legend=dict(orientation='h', y=1.1),  # Posiciona a legenda acima do gráfico
        hovermode='x',  # Exibe informações no hover apenas na posição x
        hoverlabel=dict(bgcolor='white', font_size=12),  # Define o estilo do hover
        yaxis=dict(showgrid=True, gridcolor='lightgray'),  # Adiciona linhas de grade ao eixo y
        font=dict(family='Arial, sans-serif', size=12, color='black'),  # Define a fonte do texto
    )

    fig_bar_grouped.update_traces(
        marker=dict(line=dict(color='white', width=1))  # Adiciona uma linha branca ao redor das barras
    )

    return fig_bar_grouped




**app.py**

In [43]:
# @title
#from helpers import (create_dropdown, parse_contents,
#                     make_predictions, text_intro)
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, '/content/bigmart_dash_app/styles.css'], suppress_callback_exceptions=True)

# Define um estilo com fundo branco e margem superior para as páginas
pagina_style = {"background-color": "white",
                "margin": "20px 110px 0 110px",
                'border': '1px solid #ccc',
                'border-radius': '5px',}

# Logotipo da empresa
logo_empresa = html.Div([
    html.Img(
        src='https://raw.githubusercontent.com/wanderson42/Portfolio-DS/main/datasets/Big_Mart_Sales/Big_Mart_Sales_Prediction-thumbnail-1200x1200.png',
        style={'width': '100%', 'border-radius': '5px'}
    ),
], style={"margin": "0 0 10px", "overflow": "hidden"})


# Conteúdo para a página "Sobre Este App"
def get_inicio_content():
    return html.Div([
        logo_empresa,
        html.H3("Sobre Este Web App", style={"padding-left": "10px"}),
        html.Hr(style={'border-top': '1px solid #616161'}),
        html.Div([
            html.Div(
                dcc.Markdown("#### Motivação"),
                style={"padding-left": "10px", "margin-top": "20px", "margin-bottom": "10px"}  # Adiciona margens acima e abaixo
            ),
            html.P([
                "O BigMart Sales é um conjunto de dados desenvolvido pela ",

                html.A("analytics vidhya",
                       href="https://datahack.analyticsvidhya.com/contest/practice-problem-big-mart-sales-iii/",
                       target="_blank"),
                " para previsão de vendas a partir de produtos em diferentes lojas, com base em atributos informativos, como tamanho da loja, localização, tipo de produto, preço, promoções e descontos, entre outros. "
                "O desafio é construir um modelo de aprendizado de máquina que possa prever com precisão as vendas futuras com base nessas informações. "
                "Portanto, o desenvolvimento desse dashboard é motivada pelo desejo de tornar a previsão (e consequente análise e interpretação) dos dados mais acessível, útil e interativa para diferentes tipos de usuários."
            ], style={"padding-left": "10px"})
        ]),
        dcc.Markdown(text_intro(), style={"padding-left": "10px"}),  # Renderiza o conteúdo formatado
        html.P([
            "Interessado no desenvolvimento do projeto? ",
            html.A("Clique aqui para ver o código fonte!",
                   href="https://colab.research.google.com/github/wanderson42/Portfolio-DS/blob/main/Projeto_Dash_App_BigMartSales.ipynb",
                   target="_blank"),
            " Para ter acesso ao projeto na íntegra, que contempla desde uma minuciosa análise de dados, bem como o treinamento de um modelo XGBoost confiável para previsão de vendas, e não menos importante, a construção desta aplicação web."
        ], style={"padding-left": "10px"})
    ], style={**pagina_style, "margin-top": "10px"})  # Reduz a margem superior para 5px


inicio_content = get_inicio_content()

# Conteúdo para a página "Previsões"
def get_previsoes_content():
    return html.Div([
        html.Div(style={'margin-top': '8px'}),  # Espaço adicional
        html.H3("XGBoost Regressor - Previsão de Vendas"),
        # Adicione seus gráficos de previsões aqui
        # dcc.Graph(...)

        # Guias de opções e etiquetas
        dcc.Tabs(id='tabs', value='tab-1', children=[
            dcc.Tab(label='Previsões Individuais', value='tab-1'),
            dcc.Tab(label='Previsões Multiplas', value='tab-2'),
            # dcc.Tab(label='Opção 3', value='tab-3'),
        ]),

        html.Div(id='tabs-content')

        # Adicione mais conteúdo conforme necessário
    ], style={**pagina_style, "margin-top": "10px"})

previsoes_content = get_previsoes_content()


app.layout = html.Div([
    dcc.Location(id='url', refresh=False),  # Objeto para controlar a URL do aplicativo

    dbc.NavbarSimple(
        brand=html.Img(
            src="https://raw.githubusercontent.com/wanderson42/Portfolio-DS/main/datasets/Big_Mart_Sales/turing_logo.png",
            height="60px"
        ),
        brand_href="/",  # Substitua pelo link desejado
        children=[
            dbc.NavItem(dbc.NavLink("Sobre Este App", href="/", id="nav_inicio")),
            dbc.NavItem(dbc.NavLink("DashBoard De Previsões", href="/predictions", id="nav_previsoes")),
        ],
        color="white",
        dark=False,
        style={
            "height": "80px",
            "box-shadow": "0px 2px 4px rgba(0, 0, 0, 0.1)",
            'border': '2px solid #ccc',
            'border-radius': '5px',
            'padding': '0 20px',  # Adiciona padding para ajustar a posição dos itens
        }
    ),

    # Conteúdo do NavItem
    html.Div(id="nav-content", style={"margin": "0 5px", "min-height": "calc(100vh - 90px)"}),
], style={"background-color": "#f5f5f5"})


# Callback para atualizar o conteúdo da página com base na URL
@app.callback(
    Output("nav-content", "children"),
    Output("nav_inicio", "className"),
    Output("nav_previsoes", "className"),
    Input("url", "pathname"),  # Monitora a URL atual
    prevent_initial_call=True
)
def update_nav_content(pathname):
    class_name_inicio = "nav-link"
    class_name_previsoes = "nav-link"
    if pathname == "/predictions":
        class_name_previsoes = "nav-link active"
        return previsoes_content, class_name_inicio, class_name_previsoes
    else:
        class_name_inicio = "nav-link active"
        return inicio_content, class_name_inicio, class_name_previsoes  # Página "Inicio" como padrão


# Função de callback para atualizar o conteúdo das guias
@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value')
)
def render_content(tab):

    if tab == 'tab-1':
        return html.Div([
            html.H3('Selecione cada atributo para realizar uma previsão:'),
            html.Div([
                html.Br(),
                # Adicione os Dropdowns para as features discretas

                html.Div([
                    create_dropdown("Item_Type"),
                ], style={"margin-right": "20px", "margin-bottom": "20px", "display": "inline-block", "width": "200px", "vertical-align": "top"}),

                html.Div([
                    create_dropdown("Item_Fat_Content"),
                ], style={"margin-right": "20px", "margin-bottom": "20px", "display": "inline-block", "width": "200px", "vertical-align": "top"}),

                html.Div([
                    create_dropdown("Item_Reference"),
                ], style={"margin-right": "20px", "margin-bottom": "20px", "display": "inline-block", "width": "200px", "vertical-align": "top"}),

                html.Br(),  # Quebra de linha após o terceiro dropdown

                html.Div([
                    create_dropdown("Outlet_Identifier"),
                ], style={"margin-right": "20px", "margin-bottom": "20px", "display": "inline-block", "width": "200px", "vertical-align": "top"}),

                html.Div([
                    create_dropdown("Outlet_Type"),
                ], style={"margin-right": "20px", "margin-bottom": "20px", "display": "inline-block", "width": "200px", "vertical-align": "top"}),

                html.Div([
                    create_dropdown("Outlet_Size"),
                ], style={"margin-right": "20px", "margin-bottom": "20px", "display": "inline-block", "width": "200px", "vertical-align": "top"}),

                html.Br(),  # Quebra de linha após o sexto dropdown

                html.Div([
                    create_dropdown("Outlet_Location_Type"),
                ], style={"margin-right": "20px", "margin-bottom": "20px", "display": "inline-block", "width": "200px", "vertical-align": "top"}),

                html.Div([
                    html.H6("Item Weight (0-500):"),
                    dcc.Input(id='input-Item_Weight', type='number',  value=0, min=0, max=500, step=25, style={ "width": "200px"}),
                ], style={"margin-right": "20px", "margin-bottom": "20px", "display": "inline-block", "width": "200px", "vertical-align": "top"}),

                html.Div([
                    #html.Label("Item_Visibility (0-1):"),
                    html.H6("Item_Visibility (0-1):"),
                    dcc.Input(id='input-Item_Visibility', type='number', value=0, min=0, max=1, step=0.1, style={ "width": "200px"}),
                ], style={"margin-right": "20px", "margin-bottom": "20px", "display": "inline-block", "width": "200px", "vertical-align": "top"}),

                html.Br(),  # Quebra de linha após o nono dropdown

                html.Div([
                    html.H6("Item_MRP (0-1000):"),
                    dcc.Input(id='input-Item_MRP', type='number', value=0, min=0, max=1000, step=50, style={ "width": "200px"}),
                ], style={"margin-right": "20px", "margin-bottom": "20px", "display": "inline-block",  "width": "200px", "vertical-align": "top"}),

                html.Div([
                    html.H6("Outlet_Years (0-100):"),
                    dcc.Input(id='input-Outlet_Years', type='number',  value=0, min=0, max=100, step=10, style={ "width": "200px"}),
                ], style={"margin-right": "20px", "margin-bottom": "20px", "display": "inline-block",  "width": "200px", "vertical-align": "top"})

            ]),
            # Adicione seu conteúdo da opção 1 aqui
            html.Button('Fazer Previsão',id='submit-button', n_clicks=0, style={
                'margin-top': '0px', 'background-color': '#f1863d',
                'text-align': 'center', 'padding': '5px',
                'border-radius': '5px', 'color': 'black',
                'text-decoration': 'none', 'display': 'block'  # Espaço abaixo do botão
                }),

            html.Div(id='individual-prediction')

        ])

    elif tab == 'tab-2':
        return html.Div([
            html.Div(style={'margin-top': '8px'}),  # Espaço adicional
            html.H4('Certifique-se que os dados estejam adequadamente padronizados:'),
            html.Div([
                dcc.Upload(
                    id='upload-data',
                    children=html.Div([
                        'Arraste e solte ou ',
                        html.A('selecione um arquivo .csv')
                    ]),
                    style={
                        'width': '100%',
                        'height': '60px',
                        'lineHeight': '60px',
                        'borderWidth': '1px',
                        'borderStyle': 'dashed',
                        'borderRadius': '5px',
                        'textAlign': 'center',
                        'margin': '10px 0'
                    },
                    # Permite upload de apenas um arquivo
                    multiple=False
                ),
                html.Div(id='output-data-upload'),
            ]),
            # Adicione seu conteúdo da opção 2 aqui
            html.Div(style={'margin-top': '8px'}),  # Espaço adicional
            #html.Button('Fazer Previsões', id='submit-button', n_clicks=0, style={'background-color': '#f1863d',
            #                                                                      'position': 'relative', 'left':'750px'}),

            html.Div(id='multiple-predictions')

        ])



# Callback para previsão individual
@app.callback(
    Output('individual-prediction', 'children'),
    Input('submit-button', 'n_clicks'),
    State('dropdown-Item_Type', 'value'),
    State('dropdown-Item_Fat_Content', 'value'),
    State('dropdown-Item_Reference', 'value'),
    State('dropdown-Outlet_Identifier', 'value'),
    State('dropdown-Outlet_Type', 'value'),
    State('dropdown-Outlet_Size', 'value'),
    State('dropdown-Outlet_Location_Type', 'value'),
    State('input-Item_Weight', 'value'),
    State('input-Item_Visibility', 'value'),
    State('input-Item_MRP', 'value'),
    State('input-Outlet_Years', 'value'),
    prevent_initial_call=True
)
def update_individual_prediction(n_clicks, item_type, item_fat_content, item_reference, outlet_identifier, outlet_type,
                                 outlet_size, outlet_location_type, item_weight,
                                 item_visibility, item_mrp, outlet_years):
    if n_clicks == 0:
        raise PreventUpdate

    #print(f"Button clicked: {n_clicks}")

    # Crie um dicionário com as entradas do usuário
    user_input = {
        'Item_Type': item_type,
        'Item_Fat_Content': item_fat_content,
        'Item_Reference': item_reference,
        'Outlet_Identifier': outlet_identifier,
        'Outlet_Type': outlet_type,
        'Outlet_Size': outlet_size,
        'Outlet_Location_Type': outlet_location_type,
        'Item_Weight': item_weight,
        'Item_Visibility': item_visibility,
        'Item_MRP': item_mrp,
        'Outlet_Years': outlet_years
    }

    # Faça previsões com base nas entradas do usuário
    prediction = make_predictions(user_input)
    prediction_rounded = np.round(prediction, 2)
    prediction_rounded = re.sub(r'[\[\]]', '', f"**Resultado:  \nItem_Outlet_Sales = R$ {prediction_rounded[0]:.2f}**")

    # Exiba as previsões em um elemento HTML (por exemplo, uma div)
    return html.H5(dcc.Markdown(prediction_rounded, style={'margin-top': '10px'}))



# Visualização de múltiplas previsões
html.Div([
    dcc.Graph(id='multiple-predictions'),  # Use dcc.Graph para mostrar um gráfico
    html.A('Baixar Previsões', id='download-predictions',
         download="predictions.csv", href="", target="_blank",
         style={
        'display': 'none'  # Oculta o link na página
    })

])
# ...

# callback para chamar a função parse_contents quando um arquivo .csv for carregado:
@app.callback(
    Output('output-data-upload', 'children'),
    Input('upload-data', 'contents')
)
def update_output(contents):
    if contents is None:
        raise PreventUpdate
    return parse_contents(contents)

# Callback para múltiplas previsões
@app.callback(
    Output('multiple-predictions', 'children'),
    Input('submit-button', 'n_clicks'),
    State('upload-data', 'contents')  # Adicionando o conteúdo do arquivo
)
def update_multiple_predictions(n_clicks, contents):

    if n_clicks == 0 or contents is None:
        raise PreventUpdate

    # Dados obtidos a partir do upload do usuario
    user_inputs = read_uploaded_data(contents)
    user_inputs_raw = user_inputs.copy()
    # Obtendo IDs dos itens para usar futuramente
    item_identifiers = user_inputs['Item_Identifier']

    # Fazendo previsões com base nas entradas do usuário
    predictions = make_predictions(user_inputs)
    predictions = np.round(predictions, 2)

    # Convertendo as previsões para DataFrame
    item_outlet_sales = pd.Series(predictions, name='Item_Outlet_Sales')

    # Gerando um arquivo CSV com as previsões
    df_with_predictions = pd.concat([item_identifiers, item_outlet_sales], axis=1)


    # Criando a tabela de resultados das previsões
    table = dash_table.DataTable(
        data=df_with_predictions.to_dict('records'),
        columns=[{'name': col, 'id': col} for col in df_with_predictions.columns],
        style_table={'height': '350px', 'overflowY': 'auto', 'position': 'relative'},
        id='results-table'  # Adicionando um ID à tabela para referência posterior
    )

    # Criando o gráfico Distribuição de Vendas por Categoria
    plotly_1 = plotly_sales_by_category(user_inputs_raw, item_outlet_sales)

    # Gerando link para download do CSV
    csv_string = df_with_predictions.to_csv(index=False, encoding='utf-8')
    csv_string = "data:text/csv;charset=utf-8," + quote(csv_string)


    # Ajustando o botão de download
    download_href = csv_string
    download_button = html.A('Baixar Previsões', id='download-predictions', download="predictions.csv",
                             href=download_href, target="_blank",
                             style={'background-color': '#2E9203', 'margin-top': '-60px',
                                    'display': 'block', 'width': '130px', 'text-align': 'center',
                                    'padding': '5px', 'border-radius': '5px', 'color': 'black',
                                    'text-decoration': 'none','margin-bottom': '10px'})

    plotly_2 = plotly_sales_over_time(user_inputs_raw, item_outlet_sales)

    # Retornando a estrutura do layout com a tabela, botão de download e gráficos
    return html.Div([
    html.H5('Resultado das Previsões:'),
    dbc.Row([
        dbc.Col(html.Div([table]), width=4),
        dbc.Col(html.Div([dcc.Graph(figure=plotly_1, id='sales-graph')]), width=8)
    ]),
    download_button,  # Botão de download
    dbc.Row([
        dbc.Col(html.Div([dcc.Graph(figure=plotly_2, id='second-sales-graph')]), width=12)
    ])
], style={'width': '100%'})



if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="external")


Dash app running on:


<IPython.core.display.Javascript object>

Como forma de testar a funcionalidade de multiplas previsões no aplicatico dash, segue o link de um conjunto de dados de produção que pode ser upado no app: https://raw.githubusercontent.com/wanderson42/bigmart_dash_app/main/bigmart_sales_test_pre_encoding.csv

Esses são dados finais, em oposição aos dados de treinamento ou validação, com intuito de realizar previsões ou fornecer insights através de um cenário mais próximo do uso real.

